# Assignment 5 - General Adversariel Network (GAN) Monet Painting Style Transfer

## Description

This project is a GAN based style transformation. The goal of this project is to develop a GAN model that can accurately transfer the Monet style of paintings to a photograph. The model will train on sample Monet painting images learning the Monet style.  It will then reproduce photographs in the Monet style. The data is provided by  the Kaggle competition 'I'm Somewhat of a Painter Myself' Competition and located at https://www.kaggle.com/competitions/gan-getting-started.


CycleGAN a very popular extension of the GAN architecture will be used to develope the model.  CycleGan deploys two Generator and discriminator models to create the style transfer paramters.  

Describe CycleGan

describe  generative deep learning models

Generative AI is a type of AI that uses deep learning techniques to generate new content, such as images, music, and text.

Given that the course material was a very brief, high level review of GAN theory and did not cover implementation of GAN models, this project will rely heavily on the tutorials in the references for implementation guidance.  Custom adaptations will be injected where possible to build a unique project structured around the tutorials base structures.

## Data Summary


In [ ]:
#!pip 


In [ ]:
#Set Page Width to 100%
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
#Load Required Resources
import os

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Import Data
# The .tfrec files from the Kaggle Competition site will be used

monet_file_id = tf.io.gfile.glob(str('gan-getting-started/monet_tfrec/*.tfrec'))

photo_file_id = tf.io.gfile.glob(str('gan-getting-started/photo_tfrec/*.tfrec'))

### EDA

EDA will be performed as follows:

    1. Verify Train Image Counts and Size
    2. Verify Test Image Counts and size
    3. Provide Sample Monet and photograh images

Various images were reviewed and the Monet images are assessed to be of the Monet style.  Given that the images are directly from the cmpetion data set no addtional EDA is requied

In [ ]:
# Sample Images and sizes
monet_file_id_df = pd.DataFrame(monet_file_id)
print('Monet File Summary:')
print(monet_file_id_df.head())
print(monet_file_id_df.info())

photo_file_id_df = pd.DataFrame(monet_file_id)
print('Photo FileSummary:')
print(monet_file_id_df.head())
print(monet_file_id_df.info())

In [ ]:
# Plot Example Images

monet1 = Image.imread('data/gan-getting-started/monet_jpg/0a5075d42a.jpg')
print('Monet Sample Image 1 Shape')
print(monet1.shape)
plt.imshow(monet1)
plt.title('Monet Sample Image 1')
plt.show()

monet2 = Image.imread('data/gan-getting-started/monet_jpg/0bd913dbc7.jpg')
print('Monet Sample Image 2 Shape')
print(monet2.shape)
plt.imshow(monet2)
plt.title('Monet Sample Image 2')
plt.show()

photo1 = Image.imread('data/gan-getting-started/photo_jpg/0a0c3a6d07.jpg')
print('Photo Sample Image 1 Shape')
print(photo1.shape)
plt.imshow(photo1)
plt.title('Photo Sample Image 1')
plt.show()

photo2 = Image.imread('data/gan-getting-started/photo_jpg/0a0d3e6ea7.jpg')
print('Photo Sample Image 2 Shape')
print(photo2.shape)
plt.imshow(photo2)
plt.title('Photo Sample Image 2')
plt.show()

In [ ]:
#Suppress Warning Messages Code
import warnings
warnings.filterwarnings('ignore')

### Models

Model Architecture




In [ ]:
#Split the training data
x_training_set, x_validation_set, target_train, target_validation= train_test_split(train_df['text'].values, train_df['target'].values, random_state=25, test_size=0.3, shuffle=True)


### Photo Comparison

In [ ]:
#Show Before and after images of photos


**Disuss Images**


In [1]:
### Sumbit Results

In [ ]:
#Get test set predictions
target_test = model.predict(sequence_x_test_set_padded, verbose = 1)
target_test
print('Test Shape: ', target_test.shape, '\n')

In [ ]:
#Generate submission file
test_submission = np.where(target_test <= 0.5, 0, 1)

final_submission = np.transpose(test_submission)[0]
final_submission = pd.DataFrame()
final_submission['id'] = test_df['id']
final_submission['target'] = test_submission
print(final_submission.head())

final_submission.to_csv('submission.csv', index=False)

### Conclusion

**Image Comparison**



**Dicussion**



**Submission**



### References

GAN — CycleGAN (Playing magic with pictures), https://jonathan-hui.medium.com/gan-cyclegan-6a50e7600d7

Monet CycleGAN Tutorial, https://www.kaggle.com/code/amyjang/monet-cyclegan-tutorial
Train your first CycleGAN for Image to Image Translation, https://blog.jaysinha.me/train-your-first-cyclegan-for-image-to-image-translation/

Overview of CycleGAN architecture and training, https://towardsdatascience.com/overview-of-cyclegan-architecture-and-training-afee31612a2f#:~:text=A%20CycleGAN%20is%20composed%20of,other%20transform%20zebras%20into%20horses.

Tensorflow CycleGan, https://www.tensorflow.org/tutorials/generative/cyclegan

A hands-on guide to TFRecords, https://towardsdatascience.com/a-practical-guide-to-tfrecords-584536bc786c

### Kaggle Code Refernces
CycleGAN Monet, https://www.kaggle.com/code/anubhav012/cyclegan-monet